In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,ForeignKey
from sqlalchemy import Column, Date, Integer, String,Float
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import datetime as dt

In [2]:
#Read imdb movies .csv file
movies_csv=pd.read_csv('Raw_Data\\IMDb movies.csv')

c:\hari\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#Look at the data types of all the columns
movies_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          85855 non-null  object 
 1   title                  85855 non-null  object 
 2   original_title         85855 non-null  object 
 3   year                   85855 non-null  object 
 4   date_published         85855 non-null  object 
 5   genre                  85855 non-null  object 
 6   duration               85855 non-null  int64  
 7   country                85791 non-null  object 
 8   language               85022 non-null  object 
 9   director               85768 non-null  object 
 10  writer                 84283 non-null  object 
 11  production_company     81400 non-null  object 
 12  actors                 85786 non-null  object 
 13  description            83740 non-null  object 
 14  avg_vote               85855 non-null  float64
 15  vo

In [4]:
#Checks max lenghts of all string i.e. object columns
for c in movies_csv:
    if movies_csv[c].dtype=='object':
        print(f'{c} {movies_csv[c].str.len().max()}')

imdb_title_id 10
title 196
original_title 196
year 13.0
date_published 13
genre 31
country 225.0
language 163.0
director 62.0
writer 64.0
production_company 101.0
actors 415.0
description 402.0
budget 16.0
usa_gross_income 12.0
worlwide_gross_income 14.0


In [5]:
#Year should be integer, checking which values are forcing pandas to make it string i.e. object
movies_csv['year'].unique()

array([1894, 1906, 1911, 1912, 1919, 1913, 1914, 1915, 1916, 1917, 1918,
       1920, 1921, 1924, 1922, 1923, 1925, 1926, 1935, 1927, 1928, 1983,
       1929, 1930, 1932, 1931, 1937, 1938, 1933, 1934, 1936, 1940, 1939,
       1942, 1943, 1941, 1948, 1944, 2001, 1946, 1945, 1947, 1973, 1949,
       1950, 1952, 1951, 1962, 1953, 1954, 1955, 1961, 1956, 1958, 1957,
       1959, 1960, 1963, 1965, 1971, 1964, 1966, 1968, 1967, 1969, 1976,
       1970, 1979, 1972, 1981, 1978, 2000, 1989, 1975, 1974, 1986, 1990,
       2018, 1977, 1982, 1980, 1993, 1984, 1985, 1988, 1987, 2005, 1991,
       2002, 1994, 1992, 1995, 2017, 1997, 1996, 2006, 1999, 1998, 2007,
       2008, 2003, 2004, 2010, 2009, 2011, 2013, 2012, 2016, 2015, 2014,
       2019, 2020, '2012', '2015', '2009', '2013', '2018', '2014', '2017',
       '2011', '2016', '1981', '1975', '2010', '1984', '2007', '2006',
       '2001', '2004', '1979', '2019', '1967', '1978', '2003', '2005',
       '1969', '1990', '1983', '2002', '1996', '2008'

In [5]:
#Replacing 'TV Movie 2019' to '2019'
movies_csv['year']= movies_csv['year'].replace(['TV Movie 2019'],2019)

In [6]:
#Data After Replacing
movies_csv['year'].unique()

array([1894, 1906, 1911, 1912, 1919, 1913, 1914, 1915, 1916, 1917, 1918,
       1920, 1921, 1924, 1922, 1923, 1925, 1926, 1935, 1927, 1928, 1983,
       1929, 1930, 1932, 1931, 1937, 1938, 1933, 1934, 1936, 1940, 1939,
       1942, 1943, 1941, 1948, 1944, 2001, 1946, 1945, 1947, 1973, 1949,
       1950, 1952, 1951, 1962, 1953, 1954, 1955, 1961, 1956, 1958, 1957,
       1959, 1960, 1963, 1965, 1971, 1964, 1966, 1968, 1967, 1969, 1976,
       1970, 1979, 1972, 1981, 1978, 2000, 1989, 1975, 1974, 1986, 1990,
       2018, 1977, 1982, 1980, 1993, 1984, 1985, 1988, 1987, 2005, 1991,
       2002, 1994, 1992, 1995, 2017, 1997, 1996, 2006, 1999, 1998, 2007,
       2008, 2003, 2004, 2010, 2009, 2011, 2013, 2012, 2016, 2015, 2014,
       2019, 2020, '2012', '2015', '2009', '2013', '2018', '2014', '2017',
       '2011', '2016', '1981', '1975', '2010', '1984', '2007', '2006',
       '2001', '2004', '1979', '2019', '1967', '1978', '2003', '2005',
       '1969', '1990', '1983', '2002', '1996', '2008'

In [7]:
#Convert Year to Interger from Object i.e. string
movies_csv['year']=pd.to_numeric(movies_csv['year'])

In [8]:
#verify after conversion
movies_csv['year'].dtype

dtype('int64')

In [9]:
#Date Published should be date, checking which values are forcing pandas to make it string i.e. object
movies_csv['date_published'].unique()
#Looking at it typically all values are of lenght 10 (yyyy-mm-dd)

array(['1894-10-09', '1906-12-26', '1911-08-19', ..., '2020-10-22',
       '2019-01-13', '2020-09-04'], dtype=object)

In [10]:
#Finding the value that has length greater than 10
[x for x in movies_csv['date_published'] if len(x) > 10]

['TV Movie 2019']

In [11]:
#Finding the movie with publised data of 'TV Movie 2019'
movies_csv[movies_csv['date_published']=='TV Movie 2019']

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
83917,tt8206668,Bad Education,Bad Education,2019,TV Movie 2019,"Biography, Comedy, Crime",108,USA,English,Cory Finley,...,"Hugh Jackman, Ray Romano, Welker White, Alliso...",The beloved superintendent of New York's Rosly...,7.1,23973,NaN,NaN,NaN,79.0,135.0,98.0


In [12]:
#By google search release date of "Bad Education" was 8th Sept 2019
#Replacing the date to 8th Sept 2019
movies_csv['date_published']= movies_csv['date_published'].replace(['TV Movie 2019'],'2019-09-08')

In [13]:
#Convert date_published to Date from Object i.e. string
#movies_csv['date_published']=pd.to_datetime(movies_csv['date_published'],format='%Y-%m-%d').dt.date
movies_csv['date_published']=pd.to_datetime(movies_csv['date_published'],format='%Y-%m-%d')

In [14]:
#Verify datatype after conversion
movies_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   imdb_title_id          85855 non-null  object        
 1   title                  85855 non-null  object        
 2   original_title         85855 non-null  object        
 3   year                   85855 non-null  int64         
 4   date_published         85855 non-null  datetime64[ns]
 5   genre                  85855 non-null  object        
 6   duration               85855 non-null  int64         
 7   country                85791 non-null  object        
 8   language               85022 non-null  object        
 9   director               85768 non-null  object        
 10  writer                 84283 non-null  object        
 11  production_company     81400 non-null  object        
 12  actors                 85786 non-null  object        
 13  d

In [15]:
#declarative Base
Base = declarative_base()

In [16]:
#Create Engine movies_db
engine = create_engine("sqlite:///movies_db.sqlite")

In [17]:
# This creates the DB (movies_dv.sqllite) and connects to it
conn = engine.connect()

In [18]:
#Define class i.e table to be created in DB
class Movies(Base):
    __tablename__='movies'
    imdb_title_id = Column(String(15),primary_key=True)
    title = Column(String(200))
    original_title = Column(String(200))
    year = Column(Integer)
    date_published = Column(Date)
    genre = Column(String(35))
    duration = Column(Integer)
    country = Column(String(255))
    language = Column(String(165))
    director = Column(String(65))
    writer = Column(String(65))
    production_company = Column(String(105))
    actors = Column(String(420))
    description = Column(String(405))
    avg_vote = Column(Float)
    votes = Column(Integer)
    budget = Column(String(20))
    usa_gross_income = Column(String(15))
    worlwide_gross_income = Column(String(15))
    metascore = Column(Float)
    reviews_from_users = Column(Float)
    reviews_from_critics = Column(Float)

In [19]:
#Creates 'movies' table in DB
Base.metadata.create_all(engine)

In [20]:
#Inserts data from movies_csv to movies table in movies_db sqllite DB.
movies_csv.to_sql('movies',conn,index=False,if_exists='append')

In [21]:
#Close Connection
conn.close()